### 一. Original Hard-marginal SVM

+ 动机：找一条胖胖的分割线，使得算法的VC维低，即所得模型的通用性高，不容易过拟合（相对于PLA）

+ 抽象为最优化问题，并将公式一步步转化为容易解决的二次规划问题

$$\max\limits_{w,b} fatness(w, b)$$
$$s.t.$$
$$y_n (w^T x_n + b) \geq 0$$
$$fatness(w, b) = \min\limits_{0 \leq n < N} distance(x_n, w, b) $$

$$\Downarrow margin \leftarrow fatness, 展开distance, 且意识到y_n (w^T x_n + b) =|w^T x_n + b|$$

$$\max \limits_{w,b} margin(w, b)$$
$$s.t.$$
$$margin(w, b) = \min\limits_{0 \leq n < N} \frac{1}{||w||} y_n (w^T x_n + b) $$

$$\Downarrow 利用平面的放缩不变性，使得margin(w,b) = \frac{1}{||w||} $$

$$\max \limits_{w,b} \frac{1}{||w||} $$
$$s.t.$$
$$\min\limits_{0 \leq n < N}  y_n (w^T x_n + b) = 1$$

$$\Downarrow 放松限制条件，将最大化变最小化，点积代替取模运算 $$

$$\min \limits_{w,b} \frac{1}{2} w^T w $$
$$s.t.$$
$$y_n (w^T x_n + b) \geq 1$$

    上述放松限制条件的操作之所以有效，可以用反证法证明在放松的限制条件下得到的w和b和原问题一致．上述问题可以直接丢到现成的二次规划算法里面求解．上述分类平面是线性的，若要实现非线性的效果，可以对原始特征进行转化（到高维空间）．下面就用解二次规划的方法来实现SVM，其中所用Solver来自CVXOPT包：

$$ \min \frac{1}{2}u^T Q u + p^T u $$
$$ s.t. $$
$$ G u \leq h $$
$$ A u = b $$

$$ u = \left[ \begin{array}{c} b \\ w \\ \end{array} \right], Q = \left[ \begin{array}{cc} 0 & 0_{1*d} \\ 0_{d*1} & I_{d*d} \end{array} \right], p = 0_{(d+1)*1}, G_{n*(d+1)} = \left[ \begin{array}{c} . \\ . \\ -y_n [1, x_n^T] \\ \end{array} \right], h = -1_{n}$$

$$ cvxopt.solvers.qp(Q, p, [, G, h [, A, b]]) $$

In [45]:
from sklearn import datasets
import numpy as np
from cvxopt import matrix, solvers

iris = datasets.load_iris()
iris = zip(iris.data, iris.target)
iris_01 = filter(lambda (X, y): y == 0 or y == 1, iris)
X = np.array(map(lambda (X, y): X, iris_01))
Y = np.array(map(lambda (X, y): 1 if y == 0 else -1, iris_01))
Y.shape = (len(X), 1)

In [49]:
%%time
d = X.shape[1]
n = X.shape[0]
Q = np.eye(d + 1)
Q[0][0] = 0
p = np.zeros((d + 1, 1))
G = np.hstack((np.ones((n, 1)), X)) * -Y
h = - np.ones((n, 1))

Q = matrix(Q)
p = matrix(p)
G = matrix(G)
h = matrix(h)

sol = solvers.qp(Q, p, G, h)

b = sol['x'][0]
w = sol['x'][1:]
print '-' * 100
print 'b:'
print b
print 'w:'
print w

     pcost       dcost       gap    pres   dres
 0:  2.4812e-01  1.0145e+01  2e+02  2e+00  2e+02
 1:  1.7120e+00 -1.5715e+01  2e+01  1e-01  2e+01
 2:  1.3950e+00 -3.9311e-01  2e+00  5e-03  8e-01
 3:  7.6712e-01  4.7966e-01  3e-01  5e-04  7e-02
 4:  8.1919e-01  5.6098e-01  3e-01  3e-04  5e-02
 5:  7.5770e-01  7.0585e-01  5e-02  3e-05  5e-03
 6:  7.4821e-01  7.4619e-01  2e-03  2e-07  2e-05
 7:  7.4806e-01  7.4804e-01  2e-05  2e-09  2e-07
 8:  7.4806e-01  7.4806e-01  2e-07  2e-11  2e-09
Optimal solution found.
----------------------------------------------------------------------------------------------------
b:
1.45056228325
w:
[-4.60e-02]
[ 5.22e-01]
[-1.00e+00]
[-4.64e-01]

CPU times: user 3.54 ms, sys: 1.5 ms, total: 5.05 ms
Wall time: 3.89 ms


二. Dual Hard-marginal SVM

+ 动机：原始二次规划问题的复杂度跟特征的维度有关系，即效率受限于特征的维度．想办法去除这一限制，以便可以对特征进行任意扩充，甚至可以扩充到无限维．